In [ ]:
!pip install openai chromadb tiktoken
!pip install sentence-transformers faiss-cpu

In [ ]:
with open("all_songs.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

songs = [song.strip() for song in raw_text.split("\n\n") if song.strip()]
print(f"Загружено {len(songs)} песен!")

Загружено 1261 песен!


In [ ]:
!pip install -U langchain langchain-community langchain-openai faiss-cpu


In [ ]:
import os
import getpass
import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.docstore.document import Document
from openai import OpenAI

In [ ]:
# Функция для создания векторного индекса FAISS
def create_search_index(texts, chunk_size=512, chunk_overlap=200):
    splitter = RecursiveCharacterTextSplitter(
        separators=['\n\n', '\n', ' '], chunk_size=chunk_size, chunk_overlap=chunk_overlap
    )

    source_chunks = [
        Document(page_content=chunk, metadata={'chunkID': i})
        for i, chunk in enumerate(splitter.split_text("\n\n".join(texts[5:1260])))
    ]

    if source_chunks:
        embedding_model = OpenAIEmbeddings(
            model="emb-openai/text-embedding-3-small",
            openai_api_base="https://api.vsegpt.ru/v1/",
        )
        db = FAISS.from_documents(source_chunks, embedding_model)
        return db, source_chunks  # Возвращаем и FAISS-базу, и список кусков
    else:
        return None, None

knowledge_base_index, source_chunks = create_search_index(songs)

In [ ]:
len(knowledge_base_index.index_to_docstore_id)

929

In [ ]:
existing_text = source_chunks[0].page_content  # Берём текст из базы
docs = knowledge_base_index.similarity_search(existing_text, k=5)
print(docs)


In [ ]:
embedding_model = OpenAIEmbeddings(
    model="emb-openai/text-embedding-3-small",
    openai_api_base="https://api.vsegpt.ru/v1/",
)

query = "Я иду по улице и вижу фонари"
query_vector = embedding_model.embed_query(query)
docs = knowledge_base_index.similarity_search_by_vector(query_vector, k=5)
print(docs)


[Document(id='711e8a33-0974-4576-898b-41e9494ea828', metadata={'chunkID': 4146}, page_content='Я, знаешь, запомню я\nИ я увидел как взрывается булик смешной под Находкой\nНа Королевской улице парами нарки гарной походкой\nКанатоходцу было грустно окаянными дорожками\nКогда говно, когда искусство\nЯ схожу за мороженным'), Document(id='fbd95441-9130-4edb-b2b9-a1a61bd72d64', metadata={'chunkID': 2635}, page_content='А он смотрит — нет, нет\nЯ иду — след, след\nВпереди — смерть, смерть\nПодожди, звоню соседям\nВоля тараканья твоя моё внимание\nБыло бы, если бы воткнула бы водочку. Поругивал\nИ, когда закусывал, весело подмигивал\nНогами двигал в такт лимба'), Document(id='6189caeb-399d-4dd4-922c-245cb3fff275', metadata={'chunkID': 3387}, page_content='И я не знаю, мне щас так стыдно на всё это смотреть, но уже не стыдно, уже нормально\nМне всегда казалось, то что у меня нет стиля\nЯ помню раньше ходила по улице, и, ну типа, вот это же важно для девушки - как она выглядит, одевается, всё та

In [ ]:
def generate_song(prompt, search_db):
    docs = search_db.similarity_search(prompt, k=10)
    similar_texts = [doc.page_content for doc in docs]

    if not similar_texts:
        similar_texts = ["Пока база пустая, но ты все равно можешь выдать жесткий куплет."]
    context = "\n".join(similar_texts)

    messages = [
        {"role": "system", "content": "Ты баттл-рэпер Слава КПСС. Используй агрессивные панчлайны, многосложные рифмы и неожиданные повороты в тексте. Важно, чтобы были рифмы"},
        {"role": "user", "content": f"Вот примеры строк:\n{context}\n\nТеперь продолжи: {prompt}"}
    ]

    completion = client.chat.completions.create(
        model="gpt-4-turbo",
        messages=messages,
        temperature=0.9
    )

    return completion.choices[0].message.content


In [ ]:
print(generate_song("Прощай антихайп", knowledge_base_index))

asalem
Прощай антихайп, парадокса черта,   
Сожгу на алтаре, как папиросу ветра.  
В битвах без правил моё слово — калаш,   
Ты выдыхай, брат, а я добавлю газ.   

Слава КПСС, здесь флаг мой взвивается,   
Рифмами режу, как сталью кривизна.  
Антигерои в маске — ярый изврат,   
Я же чист, как слеза, в этом рэпе марафон.   

Уже в игре, где каждый кадр на счету,   
Твоя ошибка, что меня за шута принял.  
Этот стиль не для слабых, это клинок и плеть,   
Моя дорога с кровью, но светит ведь рассвет.   

Ты просто ноль, я — бесконечная единица,   
Отдай дань уважения, ведь я здесь виновник пира.  
Мой текст — это мощь, а твой — суета,   
Я король антихайпа, а ты лишь суета.   

Забудь про шансы, я разрушу твой картон,   
Слава здесь не шутит, это не потёмкинское дно.  
Антихайп мой мундир, я в нем как в броне,   
Своими словами я зажгу звезды на дне.


In [ ]:
print(generate_song("Прощай антихайп", knowledge_base_index))

asalem
Прощай, антихайп, ты банный гиперлуп, уходишь в никуда,  
Словно флешка без файлов, ты теперь уже не мода.  
Прощай, антихайп, ты мои сны не качаешь,  
В моих строчках, как в океане, ты безвольно утопаешь.

Я на микро, как на троне, король стиля и рифм,  
Твой антихайп — просто шум, мой рэп — это гимн.  
Я разрываю сцену, словно торнадо, вихрь,  
Там, где ты не смог, я за секунду рифмы пришил.

С каждым битом, словно молот, я кую мой путь,  
А твои слова, как пушинки, просто ветром сдуло вдруг.  
Ты живешь в мире шаблонов, я — создатель новизны,  
Твой антихайп застыл в прошлом, я рву к будущему мосты.

Прощай, антихайп, ты сгорел, как старая свеча,  
Мои строки — живой огонь, что сжигает все без веса.  
Твой поток слов без содержания, мой — смысловая глубина,  
Я как сталь, тверд и ясен, а ты — как туман, иди спина к спине.

Запомни, антихайп, в этом ринге я победитель,  
Словно скала против песка, я стою здесь вертеп верный.  
Ты уходишь на закат, я же восхожу на рассвете,  
С